## Question

7) How accurately can we predict the length of a conversation with ChatGPT based on the initial prompt and context provided?

### Data extraction

In [9]:
import pandas as pd
from pandas import json_normalize

# Definir rutas de los archivos
file_discussion = "../data/snapshot_20230831/20230831_061926_discussion_sharings.json"
file_issue = "../data/snapshot_20230831/20230831_061759_issue_sharings.json"

# Cargar los archivos JSON
discussion_df = pd.read_json(file_discussion)
issue_df = pd.read_json(file_issue)

# Normalizar la columna 'Sources'
discussion_sources = json_normalize(discussion_df['Sources'])
issue_sources = json_normalize(issue_df['Sources'])

# Explorar y normalizar 'ChatgptSharing'
discussion_gptsh = json_normalize(discussion_sources['ChatgptSharing'].explode())
issue_gptsh = json_normalize(issue_sources['ChatgptSharing'].explode())

# Agregar columna identificadora 'conversation_id'
discussion_gptsh['conversation_id'] = (
    discussion_gptsh['Title'].fillna('Unknown') + '_' + discussion_gptsh['Mention.MentionedURL']
)
issue_gptsh['conversation_id'] = (
    issue_gptsh['Title'].fillna('Unknown') + '_' + issue_gptsh['Mention.MentionedURL']
)

# Reemplazar valores nulos en 'Conversations' con listas vacías
discussion_gptsh['Conversations'] = discussion_gptsh['Conversations'].apply(lambda x: x if isinstance(x, list) else [])
issue_gptsh['Conversations'] = issue_gptsh['Conversations'].apply(lambda x: x if isinstance(x, list) else [])

# Normalizar la columna 'Conversations'
discussion_conversations = json_normalize(discussion_gptsh['Conversations'].explode())
issue_conversations = json_normalize(issue_gptsh['Conversations'].explode())

# Propagar 'conversation_id' a los datos normalizados
discussion_conversations['conversation_id'] = discussion_gptsh['conversation_id'].repeat(
    discussion_gptsh['Conversations'].apply(len)
).reset_index(drop=True)

issue_conversations['conversation_id'] = issue_gptsh['conversation_id'].repeat(
    issue_gptsh['Conversations'].apply(len)
).reset_index(drop=True)

# Mostrar información sobre los DataFrames procesados
print("\nDiscussion Conversations Info:")
discussion_conversations.info()

print("\nIssue Conversations Info:")
issue_conversations.info()



Discussion Conversations Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Prompt           186 non-null    object
 1   Answer           186 non-null    object
 2   ListOfCode       186 non-null    object
 3   conversation_id  186 non-null    object
dtypes: object(4)
memory usage: 6.1+ KB

Issue Conversations Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Prompt           1747 non-null   object
 1   Answer           1747 non-null   object
 2   ListOfCode       1747 non-null   object
 3   conversation_id  1747 non-null   object
dtypes: object(4)
memory usage: 55.8+ KB


In [10]:
import pandas as pd
from pandas import json_normalize

# Definir rutas de los archivos
file_discussion = "../data/snapshot_20230831/20230831_061926_discussion_sharings.json"
file_issue = "../data/snapshot_20230831/20230831_061759_issue_sharings.json"

# Cargar los archivos JSON
discussion_df = pd.read_json(file_discussion)
issue_df = pd.read_json(file_issue)

# Normalizar la columna 'Sources'
discussion_sources = json_normalize(discussion_df['Sources'])
issue_sources = json_normalize(issue_df['Sources'])

print("\nDiscussion Sources Info:")
discussion_sources.info()
print("\nIssue Sources Info:")
issue_sources.info()

# Explorar y normalizar 'ChatgptSharing'
discussion_gptsh = json_normalize(discussion_sources['ChatgptSharing'].explode())
issue_gptsh = json_normalize(issue_sources['ChatgptSharing'].explode())

print("\nDiscussion ChatgptSharing Info:")
discussion_gptsh.info()
print("\nIssue ChatgptSharing Info:")
issue_gptsh.info()



Discussion Sources Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Type            45 non-null     object
 1   URL             45 non-null     object
 2   Author          45 non-null     object
 3   RepoName        45 non-null     object
 4   RepoLanguage    39 non-null     object
 5   Number          45 non-null     int64 
 6   Title           45 non-null     object
 7   Body            45 non-null     object
 8   CreatedAt       45 non-null     object
 9   ClosedAt        4 non-null      object
 10  UpdatedAt       45 non-null     object
 11  Closed          45 non-null     bool  
 12  UpvoteCount     45 non-null     int64 
 13  ChatgptSharing  45 non-null     object
dtypes: bool(1), int64(2), object(11)
memory usage: 4.7+ KB

Issue Sources Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (